In [9]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [10]:
video = cv2.VideoCapture("road_car_view.mp4")

In [11]:
def make_coord(frame,para):
    slope,inter=para
    y1=frame.shape[0]
    y2=int(y1*(3/5))
    x1=int((y1-inter)/slope)
    x2=int((y2-inter)/slope)
    return np.array([x1,y2,x2,y2])

In [12]:
def avg_slope_inter(frame,lines):
    left_fit=[]
    right_fit=[]
    for line in lines:
        x1,y1,x2,y2=line.reshape(4)
        para=np.polyfit((x1,x2),(y1,y2),1)
        slope=para[0]
        inter=para[1]
        if slope<0:#something wrong here
            left_fit.append((slope,inter))
        else:
            right_fit.append((slope,inter))
    if left_fit:
        left_fit_av=np.average(left_fit,axis=0)
        left_line=make_coord(frame,left_fit_av)
    if right_fit:
        right_fit_av=np.average(right_fit,axis=0)
        right_line=make_coord(frame,right_fit_av)
    return np.array([left_line,right_line])


In [13]:
def cannyfunc(frame):
    gray = cv2.cvtColor(frame,cv2.COLOR_RGB2GRAY) #gray
    blur = cv2.GaussianBlur(gray,(5,5),0)
    #lower_y=np.array([18,94,140])
    #upper_y=np.array([48,255,255])
    #mask = cv2.inRange(blur,lower_y,upper_y)
    canny=cv2.Canny(blur,50,150) #maybe 50,150
    return canny

In [14]:
def roi(frame):
    height = frame.shape[0]
    width = frame.shape[1]
    polygon = np.array([[(0,height),(width,height),(width,600),(700,500),(0,500)]])
    mask = np.zeros_like(frame)
    cv2.fillPoly(mask,polygon,255)
    masked_frame=cv2.bitwise_and(frame,mask)
    return masked_frame

In [15]:
def display_lines(frame,lines):
    line_frame=np.zeros_like(frame)
    if lines is not None:
        for line in lines:
            x1,y1,x2,y2=line.reshape(4)
            cv2.line(line_frame,(x1,y1),(x2,y2),(255,0,0),10)
    return line_frame

In [16]:
while True:
    ret, or_frame = video.read()
    if not ret:
        video=cv2.VideoCapture("road_car_view.mp4")
        continue
    lane_frame = np.copy(or_frame)
    canny = cannyfunc(lane_frame)
    cropped_frame=roi(canny)
    lines=cv2.HoughLinesP(cropped_frame,2,np.pi/180,100,np.array([]),minLineLength=40,maxLineGap=5)
    avg_lines = avg_slope_inter(lane_frame,lines)
    line_frame = display_lines(or_frame,avg_lines)
    combo=cv2.addWeighted(lane_frame,0.8,line_frame,1,1)
    cv2.imshow("result",line_frame)
    cv2.imshow("result2", combo)
    key=cv2.waitKey(25)
    if(key==27):
        break

UnboundLocalError: local variable 'left_line' referenced before assignment